In [ ]:
# Homework 1:

# Using chinook.db write pandas code.

# Customer Purchases Analysis:
# Find the total amount spent by each customer on purchases (considering invoices).
# Identify the top 5 customers with the highest total purchase amounts.
# Display the customer ID, name, and the total amount spent for the top 5 customers.
# Album vs. Individual Track Purchases:
# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
# A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
# Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).



In [ ]:
import pandas as pd
import sqlite3
connection =  sqlite3.connect("chinook.db")
sql_query = "SELECT name From sqlite_master Where type ='table' "

tables = pd.read_sql_query(sql_query, connection)["name"].tolist()
tables




['albums',
 'sqlite_sequence',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks',
 'sqlite_stat1']

In [ ]:
# Customer Purchases Analysis:
# Find the total amount spent by each customer on purchases (considering invoices).
# Identify the top 5 customers with the highest total purchase amounts.
# Display the customer ID, name, and the total amount spent for the top 5 customers.
dfs =  {table: pd.read_sql_query(f"select * from {table}", connection) for table in tables}
dfs["customers"]
dfs["invoice_items"]
dfs["invoices"]
cust_inv = dfs["customers"].merge(dfs["invoices"], on = "CustomerId", how = "inner")
cust_inv_items = cust_inv.merge(dfs["invoice_items"], on = "InvoiceId", how = "inner")
cust_inv_items["Spend by Customers"] = cust_inv_items.UnitPrice*cust_inv_items.Quantity
total_spent =  cust_inv_items.groupby(["CustomerId", "FirstName"]).agg( total_spend = pd.NamedAgg(column="Spend by Customers", aggfunc="sum")).reset_index()
top_customers = total_spent.sort_values("total_spend", ascending=False).head()
top_customers


,CustomerId,FirstName,total_spend
5,6,Helena,49.62
25,26,Richard,47.62
56,57,Luis,46.62
44,45,Ladislav,45.62
45,46,Hugh,45.62


In [ ]:
# Album vs. Individual Track Purchases:
# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
# A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
# Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).
# print(dfs["tracks"].columns,
# dfs["albums"].columns,
# dfs["customers"].columns,
# dfs["invoice_items"].columns,
# dfs["sqlite_sequence"].columns,
# dfs["employees"].columns,
# dfs["genres"].columns,
# dfs["invoices"].columns,
# dfs["media_types"].columns,
# dfs["playlists"].columns,
# dfs["playlist_track"].columns,
# dfs["sqlite_stat1"].columns)
invoice_items = dfs["invoice_items"]
tracks = dfs["tracks"]
customers = dfs["customers"]
invoices = dfs["invoices"]

items_tracks = invoice_items.merge(tracks, on="TrackId")[["InvoiceId", "AlbumId", "TrackId"]]

album_sizes = tracks.groupby("AlbumId")["TrackId"].count().reset_index().rename(columns={"TrackId": "AlbumTrackCount"})

items_tracks = items_tracks.merge(album_sizes, on="AlbumId")

purchased_album_tracks = items_tracks.groupby(["InvoiceId", "AlbumId"])["TrackId"].count().reset_index().rename(columns={"TrackId": "TracksBought"})

purchased_album_tracks = purchased_album_tracks.merge(album_sizes, on="AlbumId")

purchased_album_tracks["IsFullAlbum"] = purchased_album_tracks["TracksBought"] == purchased_album_tracks["AlbumTrackCount"]

purchased_album_tracks = purchased_album_tracks.merge(invoices[["InvoiceId", "CustomerId"]], on="InvoiceId")

customer_purchase_type = purchased_album_tracks.groupby("CustomerId")["IsFullAlbum"].any().reset_index()

only_tracks = customer_purchase_type[customer_purchase_type["IsFullAlbum"] == False]

percentage_tracks = len(only_tracks) / len(customers) * 100
print(f"{percentage_tracks:.2f}% of customers only buy individual tracks.")

64.41% of customers only buy individual tracks.


In [ ]:
import pandas as pd

# 1. Count how many tracks are in each album
album_track_counts = dfs["tracks"].groupby("AlbumId")["TrackId"].nunique().reset_index(name="total_tracks")

# 2. Count how many unique tracks each customer bought from each album
cust_album_tracks = (
    dfs["invoice_items"]
    .merge(dfs["invoices"], on="InvoiceId")                   # get CustomerId
    .merge(dfs["tracks"][["TrackId", "AlbumId"]], on="TrackId")  # get AlbumId
    .groupby(["CustomerId", "AlbumId"])["TrackId"]
    .nunique()
    .reset_index(name="tracks_bought")
)

# 3. Compare bought tracks vs total album tracks
cust_album_tracks = cust_album_tracks.merge(album_track_counts, on="AlbumId")
cust_album_tracks["preference"] = cust_album_tracks.apply(
    lambda row: "Full Album" if row["tracks_bought"] == row["total_tracks"] else "Individual Tracks",
    axis=1
)

# 4. Collapse to customer level:
# If a customer ever bought a full album → mark them "Full Album", else "Individual Tracks"
customer_pref = (
    cust_album_tracks.groupby("CustomerId")["preference"]
    .apply(lambda prefs: "Full Album" if "Full Album" in prefs.values else "Individual Tracks")
    .reset_index()
)

# 5. Calculate percentages
summary = customer_pref["preference"].value_counts(normalize=True) * 100
print(summary)




preference
Individual Tracks    64.40678
Full Album           35.59322
Name: proportion, dtype: float64
